In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Freeform Curves
A curve is often assumed to be a freeform curve that is drawn by placing control points and shaped by moving these points around. 

This general approach to curve-making includes a number of distinct classes of parametric curves each defined by its own function. These include:
* Bézier
* B-spline
* NURBS

These are all functions based on ***interpolation methods***, which share the common feature that control points are used as input and curve points are generated somewhere in between.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P23.jpg" style="width: 200px; display: inline;">

## Interpolating with Polynomial Functions

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P19.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P20.jpg" style="width: 200px; display: inline;">

## Interpolating with Piecewise Polynomial Functions

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P21.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P22.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Curve Through Points
Given a set of control Points cpts and a tension value ten, constructs a Curve by hermite cubic
interpolation with tension control
"""

# sum dists between control pts and construct intervals
ivals = []
sum = 0
for dist in [pa.dist(pb) for pa,pb in zip(cpts[:-1],cpts[1:])]:
    ivals.append(Interval(sum,sum+dist))
    sum += dist

# add tangent control points
cpts.insert( 0, cpts[0]+Vec(cpts[1],cpts[0]) )
cpts.append( cpts[-1]+Vec(cpts[-2],cpts[-1]) )

# a span relates an interval to a set of points
spans = {}
for n, ival in enumerate(ivals):
    spans[ival] = [ cpts[n], cpts[n+1], cpts[n+2], cpts[n+3] ]

def func(t):
    t = t*sum
    for ival in spans:
        if t in ival:
            # set t to the normalized value for this span
            t = ival.deval(t)
            # square and cube of t
            t2, t3 = t**2, t**3
            # the points of the span
            p0,p1,p2,p3 = spans[ival]
            
            # hermite interpolation
            tau = 0.5*(1-ten)*(1+bai)
            m0, m1 = (p2-p0)*tau, (p3-p1)*tau
            a0 = 2*t3 - 3*t2 + 1
            a1 = t3 - 2*t2 + t
            a2 = t3 - t2
            a3 = -2*t3 + 3*t2
            return( p1*a0 + m0*a1 + m1*a2 + p2*a3 )
            
crv = Curve(func)

## Interpolating with Rational Functions, Enter NURBS

In [ ]:
def bezier(cpts):
    def func(t):
        pts = cpts
        while len(pts) > 1:
            pts = [Point.interpolate(pts[n],pts[n+1],t) for n in range(len(pts)-1)]
        return pts[0]
    return Curve(func)